In [8]:
# Para calcular las features contextuales hay que partir de años anteriores
# En este caso y entendiento el avance de la F1, he dejado 3 años como contexto anterior para estas variables
# Debido a esto los años 2005, 2006 y 2007 no se van a poder utilizar en el entrenamiento, ya que es necesario tener todas las variables disponibles para entrenar al modelo

In [9]:
# He decidido poner variables dinamicas para que el modelo vaya entendiendo la tendencia y el contexto de cada piloto en cada momento de la temporada
# De ese modo de podrá hacer una predicción mas acertada, ademas de que con estas features se puede ir actualizando ronda a ronda las predicciones ya que dependen de una fuente actualizada como es FastF1

In [10]:
# La primera variable que pienso que es importante para el modelo es la calidad del piloto
# Debido a que no hay una regla para este calculo, propongo tre ejes sobre los cuales calcularlo: 

'''
1. Performance Z-Score (rendimiento vs parrilla)

z_score = (posicion_media_piloto - posicion_media_parrilla) / desviacion_parrilla

Ejemplo Verstappen 2024:
- Posición media: 1.4
- Media parrilla: 10.5
- Desviación: 4.2
→ z_score = (1.4 - 10.5) / 4.2 = -2.17 (muy negativo = muy bueno)

2. Gap vs Compañero (domina al compañero)

gap_z = z_score_piloto - z_score_compañero

Verstappen vs Pérez 2024:
- Verstappen z = -2.17
- Pérez z = -0.43
→ gap_z = -2.17 - (-0.43) = -1.74 (Verstappen domina)

3. Consistency (volatilidad de resultados)

cv = desviacion_posiciones / media_posiciones

Verstappen 2024:
- Posiciones: [P1, P1, P2, P1, P3, P1, ...]
- CV = 0.50 (muy consistente)

'''

# Como no se puede calcular de esta forma a todos los pilotos, he decidido asignar a cada equipo un peso diferente en cada una de estas variables
# Esto se debe a que no es lo mismo hacerlo bien o ser consistente en un equipo mejor que en uno peor


'''
Top 4 equipos (Red Bull, Ferrari, Mercedes, McLaren):
  - Performance: 45%
  - Gap compañero: 40%  
  - Consistency: 15%

Midfield (Aston Martin, Alpine, Racing Bulls):
  - Performance: 50%
  - Gap compañero: 30%
  - Consistency: 20%

Backmarkers (Williams, Haas, Sauber):
  - Performance: 55%
  - Gap compañero: 20% 
  - Consistency: 25%

'''

# Además para normalizar el ruido, voy a ponderar estos calculos en los ultimos 3 años, asignando mas importancia a los mas recientes

'''
driver_quality_2025 = 0.60 × quality_2024  # 60% año reciente
                    + 0.30 × quality_2023  # 30% año anterior
                    + 0.10 × quality_2022  # 10% hace 2 años
Ejemplo Verstappen:
- 2024: 97.3
- 2023: 98.1
- 2022: 96.5
→ driver_quality_2025 = 0.60×97.3 + 0.30×98.1 + 0.10×96.5 = 97.5
'''

'\ndriver_quality_2025 = 0.60 × quality_2024  # 60% año reciente\n                    + 0.30 × quality_2023  # 30% año anterior\n                    + 0.10 × quality_2022  # 10% hace 2 años\nEjemplo Verstappen:\n- 2024: 97.3\n- 2023: 98.1\n- 2022: 96.5\n→ driver_quality_2025 = 0.60×97.3 + 0.30×98.1 + 0.10×96.5 = 97.5\n'

In [11]:
# Una de lass cosas mas predictoras del resulatdo en F1 es el coche que se va a conducir, por ello incluyo variables de la mejoria o no del coche en los ultimos años y la posicion promedio del equipo
# Estas features seran muy importantes a principo de temporada ya que hay pocos datos

'''
Promedio ponderado de las posiciones del equipo en 3 años

Red Bull Racing:
- 2024: 1.8 (posición media de Verstappen + Pérez)
- 2023: 2.1
- 2022: 2.5
→ team_avg_pos_3y = 0.65×1.8 + 0.25×2.1 + 0.10×2.5 = 2.0
'''


'''
team_trend = team_avg_2024 - team_avg_2023

McLaren:
- 2024: 6.1
- 2023: 9.9
→ team_trend = 6.1 - 9.9 = -3.8 (MEJORANDO, negativo es bueno)

'''

'\nteam_trend = team_avg_2024 - team_avg_2023\n\nMcLaren:\n- 2024: 6.1\n- 2023: 9.9\n→ team_trend = 6.1 - 9.9 = -3.8 (MEJORANDO, negativo es bueno)\n\n'

In [12]:
# Para poder hacer todos estos calculos es necesario mapear el nombre de los equipos, ya que estos suelen variar por patrocinios u otros motivos aunque el equipo sea el mismo
# Es necesario para tener una consistencia a lo largo del tiempo

In [13]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from scipy.stats import percentileofscore


INPUT_FILE = '../data/processed/f1_clean.csv'
OUTPUT_FILE = '../data/processed/context_stats_rolling.pkl'


TEAM_MAPPING = {
    # Aston Martin lineage
    'Aston Martin': 'Aston Martin',
    'Aston Martin F1 Team': 'Aston Martin',
    'Racing Point': 'Aston Martin',
    'Racing Point F1 Team': 'Aston Martin',
    'Force India': 'Aston Martin',
    'Spyker': 'Aston Martin',
    'Spyker F1': 'Aston Martin',
    'Jordan': 'Aston Martin',
    'Midland': 'Aston Martin',
    
    # Alpine lineage
    'Alpine': 'Alpine',
    'Alpine F1 Team': 'Alpine',
    'Renault': 'Alpine',
    'Lotus F1': 'Alpine',
    'Lotus': 'Alpine',
    
    # Red Bull family
    'Red Bull Racing': 'Red Bull Racing',
    'Red Bull': 'Red Bull Racing',
    
    'Racing Bulls': 'AlphaTauri',
    'RB': 'AlphaTauri',
    'AlphaTauri': 'AlphaTauri',
    'Toro Rosso': 'AlphaTauri',
    
    # Sauber lineage
    'Kick Sauber': 'Sauber',
    'Alfa Romeo': 'Sauber',
    'Sauber': 'Sauber',
    'Alfa Romeo Sauber': 'Sauber',
    'BMW Sauber': 'Sauber',
    
    # Mercedes lineage
    'Mercedes': 'Mercedes',
    'Mercedes-Benz': 'Mercedes',
    'Brawn': 'Mercedes',
    
    # Haas
    'Haas F1 Team': 'Haas',
    'Haas': 'Haas',
    
    # Stable teams
    'Ferrari': 'Ferrari',
    'McLaren': 'McLaren',
    'Williams': 'Williams',
    
    # Historic teams
    'Toyota': 'Toyota',
    'Honda': 'Honda',
    'BAR': 'BAR',
    'Jaguar': 'Jaguar',
    'Minardi': 'Minardi',
    'Arrows': 'Arrows',
    'Prost': 'Prost',
    'Benetton': 'Benetton',
    'Virgin': 'Virgin',
    'Lotus Racing': 'Lotus Racing',
    'HRT': 'HRT',
    'Marussia': 'Marussia',
    'Caterham': 'Caterham',
    'Manor': 'Manor',
}

def normalizar_team(team):
    """Normaliza nombre de equipo para continuidad histórica"""
    return TEAM_MAPPING.get(team, team)


def encontrar_compañero(df_year, driver, team):
    """Encuentra el compañero de equipo"""
    compañeros = df_year[
        (df_year['team'] == team) & 
        (df_year['driver'] != driver)
    ]['driver'].unique()
    return compañeros[0] if len(compañeros) > 0 else None

def calcular_percentil_seguro(value, all_values):
    """Calcula percentil evitando errores"""
    try:
        if len(all_values) > 0:
            return percentileofscore(all_values, value, kind='rank')
        else:
            return 50.0
    except:
        return 50.0

# ==================== DRIVER QUALITY ====================

def calcular_driver_quality_año(df, year, constructor_positions):
    """Calcula driver_quality para UN año específico"""
    
    df_year = df[df['year'] == year].copy()
    df_year['team'] = df_year['team'].apply(normalizar_team)
    
    if len(df_year) == 0:
        return pd.DataFrame(columns=['driver', 'driver_quality'])
    
    posiciones_todas = df_year[~df_year['es_abandono']]['position']
    
    if len(posiciones_todas) < 10:
        return pd.DataFrame(columns=['driver', 'driver_quality'])
    
    grid_mean = posiciones_todas.mean()
    grid_std = posiciones_todas.std()
    
    if grid_std == 0:
        grid_std = 1.0
    
    driver_stats = []
    
    for driver in df_year['driver'].unique():
        df_driver = df_year[df_year['driver'] == driver]
        team = df_driver['team'].iloc[0]
        
        # Z-SCORE
        posiciones_terminadas = df_driver[~df_driver['es_abandono']]['position']
        
        if len(posiciones_terminadas) >= 3:
            driver_mean = posiciones_terminadas.mean()
            z_score = (driver_mean - grid_mean) / grid_std
        else:
            z_score = 0.0
        
        # GAP VS COMPAÑERO
        teammate = encontrar_compañero(df_year, driver, team)
        
        if teammate and teammate in df_year['driver'].values:
            teammate_positions = df_year[
                (df_year['driver'] == teammate) &
                (~df_year['es_abandono'])
            ]['position']
            
            if len(teammate_positions) >= 3:
                teammate_mean = teammate_positions.mean()
                teammate_z = (teammate_mean - grid_mean) / grid_std
                gap_z = z_score - teammate_z
            else:
                gap_z = 0.0
        else:
            gap_z = 0.0
        
        # CONSISTENCY
        if len(posiciones_terminadas) >= 3:
            cv = posiciones_terminadas.std() / posiciones_terminadas.mean()
        else:
            cv = 1.0
        
        driver_stats.append({
            'driver': driver,
            'team': team,
            'z_score': z_score,
            'gap_z': gap_z,
            'cv': cv
        })
    
    df_stats = pd.DataFrame(driver_stats)
    
    if len(df_stats) == 0:
        return pd.DataFrame(columns=['driver', 'driver_quality'])
    
    # Percentiles
    df_stats['performance_percentile'] = df_stats['z_score'].apply(
        lambda x: calcular_percentil_seguro(-x, -df_stats['z_score'].values)
    )
    df_stats['teammate_percentile'] = df_stats['gap_z'].apply(
        lambda x: calcular_percentil_seguro(-x, -df_stats['gap_z'].values)
    )
    df_stats['consistency_percentile'] = df_stats['cv'].apply(
        lambda x: 100 - calcular_percentil_seguro(x, df_stats['cv'].values)
    )
    
    # Pesos dinámicos
    def get_weights(team):
        constructor_pos = constructor_positions.get(team, 10)
        if constructor_pos <= 4:
            return {'perf': 0.45, 'team': 0.40, 'cons': 0.15}
        elif constructor_pos <= 7:
            return {'perf': 0.50, 'team': 0.30, 'cons': 0.20}
        else:
            return {'perf': 0.55, 'team': 0.20, 'cons': 0.25}
    
    df_stats['driver_quality'] = df_stats.apply(
        lambda row: (
            get_weights(row['team'])['perf'] * row['performance_percentile'] +
            get_weights(row['team'])['team'] * row['teammate_percentile'] +
            get_weights(row['team'])['cons'] * row['consistency_percentile']
        ),
        axis=1
    )
    
    return df_stats[['driver', 'driver_quality']]

# ==================== TEAM STATS ====================

def calcular_team_stats_año(df, year):
    """Calcula team_avg_pos para UN año"""
    
    df_year = df[df['year'] == year].copy()
    df_year['team'] = df_year['team'].apply(normalizar_team)
    
    if len(df_year) == 0:
        return pd.DataFrame(columns=['team', 'team_avg_pos'])
    
    team_stats = []
    
    for team in df_year['team'].unique():
        df_team = df_year[
            (df_year['team'] == team) &
            (~df_year['es_abandono'])
        ]
        
        if len(df_team) > 0:
            team_avg = df_team['position'].mean()
        else:
            team_avg = 15.0
        
        team_stats.append({
            'team': team,
            'team_avg_pos': team_avg
        })
    
    return pd.DataFrame(team_stats)

# ==================== ROLLING WINDOW ====================

def calcular_context_stats_rolling(df, start_year=2008, end_year=2025):
    """Calcula context stats con rolling window"""
    
    context_stats_all = {}
    
    for target_year in range(start_year, end_year + 1):
        print(f"Año {target_year}...", end=" ")
        
        context_years = [target_year - 3, target_year - 2, target_year - 1]
        driver_weights = [0.10, 0.30, 0.60]
        team_weights = [0.10, 0.25, 0.65]
        
        # CONSTRUCTOR POSITIONS
        df_prev_year = df[df['year'] == target_year - 1].copy()
        df_prev_year['team'] = df_prev_year['team'].apply(normalizar_team)
        
        constructor_positions = {}
        for team in df_prev_year['team'].unique():
            team_avg = df_prev_year[
                (df_prev_year['team'] == team) &
                (~df_prev_year['es_abandono'])
            ]['position'].mean()
            constructor_positions[team] = team_avg
        
        sorted_teams = sorted(constructor_positions.items(), key=lambda x: x[1])
        constructor_positions = {team: idx+1 for idx, (team, _) in enumerate(sorted_teams)}
        
        # DRIVER QUALITY
        driver_quality_dict = {}
        
        for i, year in enumerate(context_years):
            if year >= 2005:
                dq_year = calcular_driver_quality_año(df, year, constructor_positions)
                
                for _, row in dq_year.iterrows():
                    driver = row['driver']
                    quality = row['driver_quality']
                    
                    if driver not in driver_quality_dict:
                        driver_quality_dict[driver] = []
                    
                    driver_quality_dict[driver].append((quality, driver_weights[i]))
        
        driver_quality_final = {}
        for driver, qualities in driver_quality_dict.items():
            total_weight = sum(w for _, w in qualities)
            if total_weight > 0:
                weighted_avg = sum(q * w for q, w in qualities) / total_weight
                driver_quality_final[driver] = weighted_avg
            else:
                driver_quality_final[driver] = 50.0
        
        # TEAM STATS
        team_stats_dict = {}
        
        for i, year in enumerate(context_years):
            if year >= 2005:
                ts_year = calcular_team_stats_año(df, year)
                
                for _, row in ts_year.iterrows():
                    team = row['team']
                    avg_pos = row['team_avg_pos']
                    
                    if team not in team_stats_dict:
                        team_stats_dict[team] = []
                    
                    team_stats_dict[team].append((avg_pos, team_weights[i]))
        
        team_avg_final = {}
        for team, avgs in team_stats_dict.items():
            total_weight = sum(w for _, w in avgs)
            if total_weight > 0:
                weighted_avg = sum(a * w for a, w in avgs) / total_weight
                team_avg_final[team] = weighted_avg
            else:
                team_avg_final[team] = 10.0
        
        # TEAM TREND
        team_trend = {}
        if target_year - 1 >= 2005 and target_year - 2 >= 2005:
            ts_recent = calcular_team_stats_año(df, target_year - 1)
            ts_prev = calcular_team_stats_año(df, target_year - 2)
            
            for team in ts_recent['team'].unique():
                avg_recent = ts_recent[ts_recent['team'] == team]['team_avg_pos'].values
                avg_prev = ts_prev[ts_prev['team'] == team]['team_avg_pos'].values
                
                if len(avg_recent) > 0 and len(avg_prev) > 0:
                    trend = avg_recent[0] - avg_prev[0]
                    team_trend[team] = trend
                else:
                    team_trend[team] = 0.0
        
        context_stats_all[target_year] = {
            'drivers': driver_quality_final,
            'teams': {
                'avg_pos_3y': team_avg_final,
                'trend': team_trend
            },
            'metadata': {
                'context_years': context_years,
                'target_year': target_year,
                'n_drivers': len(driver_quality_final),
                'n_teams': len(team_avg_final)
            }
        }
        
        print(f"✓ {len(driver_quality_final)} pilotos, {len(team_avg_final)} equipos")
    
    return context_stats_all


# Calcular context stats
context_stats = calcular_context_stats_rolling(df, start_year=2008, end_year=2025)


Path(OUTPUT_FILE).parent.mkdir(parents=True, exist_ok=True)

with open(OUTPUT_FILE, 'wb') as f:
    pickle.dump(context_stats, f)


Año 2008... ✓ 39 pilotos, 15 equipos
Año 2009... ✓ 38 pilotos, 13 equipos
Año 2010... ✓ 34 pilotos, 12 equipos
Año 2011... ✓ 37 pilotos, 14 equipos
Año 2012... ✓ 40 pilotos, 12 equipos
Año 2013... ✓ 37 pilotos, 13 equipos
Año 2014... ✓ 38 pilotos, 13 equipos
Año 2015... ✓ 36 pilotos, 12 equipos
Año 2016... ✓ 33 pilotos, 12 equipos
Año 2017... ✓ 33 pilotos, 13 equipos
Año 2018... ✓ 32 pilotos, 11 equipos
Año 2019... ✓ 30 pilotos, 11 equipos
Año 2020... ✓ 31 pilotos, 11 equipos
Año 2021... ✓ 29 pilotos, 11 equipos
Año 2022... ✓ 28 pilotos, 10 equipos
Año 2023... ✓ 30 pilotos, 11 equipos
Año 2024... ✓ 29 pilotos, 11 equipos
Año 2025... ✓ 28 pilotos, 10 equipos


In [17]:
df_context_stats = pd.read_pickle('../data/processed/context_stats_rolling.pkl')

In [18]:
df_context_stats

{2008: {'drivers': {'FIS': 52.18376068376068,
   'BAR': 43.74501424501425,
   'ALO': 74.54131054131054,
   'COU': 53.858974358974365,
   'WEB': 63.65242165242165,
   'MON': 65.18518518518519,
   'KLI': 39.25925925925926,
   'RAI': 78.97008547008546,
   'TRU': 50.94301994301994,
   'MAS': 50.52279202279203,
   'SCH': 50.27635327635328,
   'VIL': 42.916666666666664,
   'KAR': 37.407407407407405,
   'TMO': 39.25925925925926,
   'FRI': 30.925925925925927,
   'BUT': 64.06267806267806,
   'SAT': 40.91025641025641,
   'MSC': 80.74074074074073,
   'HEI': 69.57407407407408,
   'ALB': 43.467236467236475,
   'DAV': 38.17867317867318,
   'DLR': 36.851851851851855,
   'WUR': 40.57590557590558,
   'LIU': 37.95584045584046,
   'ZON': 40.37037037037037,
   'DOO': 39.39814814814815,
   'PIZ': 28.51851851851852,
   'ROS': 52.7920227920228,
   'SPE': 53.010446343779684,
   'IDE': 54.62962962962963,
   'FMO': 27.22222222222222,
   'YAM': 28.04368471035138,
   'KUB': 47.160493827160494,
   'HAM': 63.269230